In [2]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
import numpy as np

pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [4]:
# dataviz
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

/opt/conda/anaconda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
super_srag = spark.read.parquet('gs://ai-covid19-datalake/trusted/reclassified_srag_16-12-2021_gbt_pcr_stratsamp.parquet').persist(StorageLevel.MEMORY_ONLY)

In [7]:
super_srag.limit(3).toPandas()

,NU_NOTIFIC,CS_SEXO,DT_NASC,AGE_AT_NOTIF,AGE_GROUP,CS_GESTANT,CS_RACA,CS_ETINIA,CS_ESCOL_N,SG_UF,CO_MUN_RES,SEM_PRI,SEM_NOT,DIST_PRI_NOTIFIC,DT_SIN_PRI,SG_UF_NOT,CO_MUN_NOT,SURTO_SG,NOSOCOMIAL,AVE_SUINO,VACINA,HOSPITAL,DIST_PRI_INTERNA,SUPORT_VEN,UTI,DIST_PRI_ENTUTI,CLASSI_OUT,CRITERIO,EVOLUCAO,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,OUTRO_SIN,OUTRO_DES,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,OBES_IMC,OUT_MORBI,MORB_DESC,RAIOX_RES,DIST_PRI_RAIOX,TOMO_RES,DIST_PRI_TOMO,AMOSTRA,TP_AMOSTRA,DT_COLETA,DIST_PRI_COLETA,...,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_AVG,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_AVG,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_AVG,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_AVG,GMR_PARKS_PERCENT_1WEEK_BEFORE_AVG,GMR_TRANSIT_STATIONS_2WEEKS_AVG,GMR_GROCERY_AND_PHARMACY_2WEEKS_AVG,GMR_RETAIL_AND_RECREATION_2WEEKS_AVG,GMR_WORKPLACES_PERCENT_2WEEKS_AVG,GMR_RESIDENTIAL_PERCENT_2WEEKS_AVG,GMR_PARKS_PERCENT_2WEEKS_AVG,INMET_TEMP_C_AVG,INMET_RELATIVE_AIR_HUMIDITY_AVG,INMET_DAILY_PRECIPT_AVG,INMET_TEMP_C_1WEEK_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_AVG,INMET_DAILY_PRECIPT_1WEEK_BEFORE_AVG,INMET_TEMP_C_2WEEKS_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_AVG,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_AVG,GMR_TRANSIT_STATIONS_Q,GMR_GROCERY_AND_PHARMACY_Q,GMR_RETAIL_AND_RECREATION_Q,GMR_WORKPLACES_PERCENT_Q,GMR_RESIDENTIAL_PERCENT_Q,GMR_PARKS_PERCENT_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_PARKS_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_GROCERY_AND_PHARMACY_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_PARKS_PERCENT_2WEEKS_Q,INMET_TEMP_C_Q,INMET_RELATIVE_AIR_HUMIDITY_Q,INMET_DAILY_PRECIPT_Q,INMET_TEMP_C_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_DAILY_PRECIPT_1WEEK_BEFORE_Q,INMET_TEMP_C_2WEEKS_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_Q,ANO,CLASSI_FIN,CLASSI_FIN_source
0,77309859740,M,1968-12-02,52,7,6,4,None,4.0,BA,290730,19,20,7,2021-05-13,BA,291920,None,NaN,NaN,9.0,1,6,2,2,NaN,None,1,None,4,6,1,1,2.0,None,9,9,9,9,None,2,None,NaN,None,None,NaN,1,1,2021-05-14,1,...,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021,5,original
1,68719688318,F,1968-12-02,52,7,5,4,None,NaN,BA,292370,19,19,5,2021-05-10,BA,290520,None,2.0,9.0,NaN,1,6,2,1,6.0,None,1,2,8,6,2,5,NaN,None,1,1,5,1,None,None,None,NaN,None,1,5.0,1,1,2021-05-14,4,...,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021,5,original
2,68719818720,F,1988-06-04,32,5,5,4,None,3.0,BA,292740,19,20,7,2021-05-12,BA,292740,None,2.0,9.0,9.0,1,6,9,2,NaN,None,1,None,3,1,1,4,1.0,PROSTACAO INTENSA,9,9,9,9,34,2,None,6.0,None,1,6.0,1,1,2021-05-15,3,...,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021,5,original


In [8]:
super_srag.select('CLASSI_FIN_source').groupBy('CLASSI_FIN_source').count().show()

+-----------------+-------+
|CLASSI_FIN_source|  count|
+-----------------+-------+
|         original|2507874|
|        predicted| 882315|
+-----------------+-------+



In [38]:
filtro_obito = (F.col('EVOLUCAO') == '2') | (F.col('EVOLUCAO') == '3')
filtro_cura = (F.col('EVOLUCAO') == '1')
filter_pred = ((F.col('CLASSI_FIN') == 5) & (F.col('CLASSI_FIN_source') == 'predicted'))
filter_homem_negro = ((F.col('CS_RACA') == 4) | (F.col('CS_RACA') == 2)) & (F.col('CS_SEXO') == 'M')
filter_mulher_negra = ((F.col('CS_RACA') == 4) | (F.col('CS_RACA') == 2)) & (F.col('CS_SEXO') == 'F')
filter_homem_branco = ((F.col('CS_RACA') == 1)) & (F.col('CS_SEXO') == 'M')
filter_mulher_branca = (F.col('CS_RACA') == 1) & (F.col('CS_SEXO') == 'F')
filter_homem_indigena = ((F.col('CS_RACA') == 5)) & (F.col('CS_SEXO') == 'M')
filter_mulher_indigena = (F.col('CS_RACA') == 5) & (F.col('CS_SEXO') == 'F')

In [29]:
super_srag.filter(filter_pred).filter(filter_homem_negro).count()

108652

In [30]:
super_srag.filter(filter_pred).filter(filter_mulher_negra).count()

96319

In [31]:
super_srag.filter(filter_pred).filter(filter_homem_branco).count()

111893

In [32]:
super_srag.filter(filter_pred).filter(filter_mulher_branca).count()

111695

In [35]:
super_srag.filter(filter_pred).filter(filter_homem_indigena).count()

364

In [36]:
super_srag.filter(filter_pred).filter(filter_mulher_indigena).count()

393

In [46]:
print(super_srag.filter(filter_pred).filter(filter_homem_negro).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_homem_negro).filter(filtro_cura).count())

29295
65370


In [47]:
print(super_srag.filter(filter_pred).filter(filter_mulher_negra).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_mulher_negra).filter(filtro_cura).count())

23249
60580


In [48]:
print(super_srag.filter(filter_pred).filter(filter_homem_branco).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_homem_branco).filter(filtro_cura).count())

30021
71538


In [49]:
print(super_srag.filter(filter_pred).filter(filter_mulher_branca).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_mulher_branca).filter(filtro_cura).count())

27111
73931


In [50]:
print(super_srag.filter(filter_pred).filter(filter_homem_indigena).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_homem_indigena).filter(filtro_cura).count())

89
237


In [51]:
print(super_srag.filter(filter_pred).filter(filter_mulher_indigena).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_mulher_indigena).filter(filtro_cura).count())

75
273


In [57]:
super_srag.filter(filter_pred).count()

528063

root
 |-- PP_IF_RESUL: string (nullable = true)
 |-- CS_RACA: integer (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- CLASSI_FIN: integer (nullable = true)
 |-- EVOLUCAO: string (nullable = true)
 |-- HOSPITAL: string (nullable = true)
 |-- UTI: integer (nullable = true)
 |-- SUPORT_VEN: integer (nullable = true)
 |-- RAIOX_RES: integer (nullable = true)
 |-- TOMO_RES: string (nullable = true)



In [19]:
super_srag.select(['CS_RACA', 'CS_SEXO', 'CLASSI_FIN']).groupBy('CS_SEXO').count().show()

+-------+-------+
|CS_SEXO|  count|
+-------+-------+
|      F|1540966|
|      M|1848458|
|      I|    765|
+-------+-------+



In [55]:
filter_hospital = (F.col('HOSPITAL') == '1')
filter_suporte_vent = (F.col('SUPORT_VEN') == 1)
filter_uti = (F.col('UTI') == 1)

In [56]:
print(super_srag.filter(filter_pred).filter(filter_hospital).count())
print(super_srag.filter(filter_pred).filter(filter_suporte_vent).count())
print(super_srag.filter(filter_pred).filter(filter_uti).count())

500887
71664
136273


In [ ]:
print(super_srag.filter(filter_pred).filter(filter_hospital).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_suporte_vent).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filter_uti).filter(filtro_obito).count())

In [ ]:
CS_RACA

In [ ]:
CS_SEXO

In [ ]:
HOSPITAL

In [ ]:
UTI

In [ ]:
SUPORT_VEN

In [62]:
filter_image_exam = ((F.col('RAIOX_RES').isNotNull()) & ((F.col('RAIOX_RES') != 6) | (F.col('RAIOX_RES') != 9))) | ((F.col('TOMO_RES').isNotNull()) & ((F.col('TOMO_RES') != '6') | (F.col('TOMO_RES') != '9')))

In [65]:
super_srag.filter(filter_pred).filter(filter_image_exam).count()

398055

In [ ]:
RAIOX_RES

In [ ]:
TOMO_RES

In [79]:
super_srag.select(['PP_RES_SOR_IGA', 'PP_RES_SOR_IGM', 'PP_RES_SOR_IGG', 'PP_PCR_RESUL', 'PP_TRA_RESUL', 'PP_IF_RESUL', 'CS_RACA', 'CS_SEXO', 'CLASSI_FIN', 'EVOLUCAO', 'HOSPITAL', 'UTI', 'SUPORT_VEN', 'RAIOX_RES', 'TOMO_RES']).printSchema()

root
 |-- PP_RES_SOR_IGA: string (nullable = true)
 |-- PP_RES_SOR_IGM: string (nullable = true)
 |-- PP_RES_SOR_IGG: string (nullable = true)
 |-- PP_PCR_RESUL: string (nullable = true)
 |-- PP_TRA_RESUL: string (nullable = true)
 |-- PP_IF_RESUL: string (nullable = true)
 |-- CS_RACA: integer (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- CLASSI_FIN: integer (nullable = true)
 |-- EVOLUCAO: string (nullable = true)
 |-- HOSPITAL: string (nullable = true)
 |-- UTI: integer (nullable = true)
 |-- SUPORT_VEN: integer (nullable = true)
 |-- RAIOX_RES: integer (nullable = true)
 |-- TOMO_RES: string (nullable = true)



In [75]:
filter_if = ((F.col('PP_IF_RESUL').isNotNull()) & (F.col('PP_IF_RESUL') !=  '9'))
filter_tra = ((F.col('PP_TRA_RESUL').isNotNull()) & (F.col('PP_TRA_RESUL') !=  '9'))
filter_pcr = ((F.col('PP_PCR_RESUL').isNotNull()) & (F.col('PP_PCR_RESUL') !=  '9'))
filter_sor = ((F.col('PP_RES_SOR_IGA').isNotNull()) & (F.col('PP_RES_SOR_IGA') !=  '9')) | ((F.col('PP_RES_SOR_IGM').isNotNull()) & (F.col('PP_RES_SOR_IGM') !=  '9')) | ((F.col('PP_RES_SOR_IGG').isNotNull()) & (F.col('PP_RES_SOR_IGG') !=  '9'))

In [76]:
print(super_srag.filter(filter_pred).filter(filter_if).count())
print(super_srag.filter(filter_pred).filter(filter_tra).count())
print(super_srag.filter(filter_pred).filter(filter_pcr).count())
print(super_srag.filter(filter_pred).filter(filter_sor).count())

62677
11595
528022
527718


In [74]:
print(super_srag.filter(filter_pred).filter(filtro_obito).count())
print(super_srag.filter(filter_pred).filter(filtro_cura).count())

131191
335463


In [ ]:
PP_IF_RESUL

In [ ]:
PP_TRA_RESUL

In [ ]:
PP_PCR_RESUL

In [ ]:
PP_RES_SOR_IGA
PP_RES_SOR_IGM
PP_RES_SOR_IGG